In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings  
import math
import time
import seaborn as sn

from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import Ridge
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from mpl_toolkits.mplot3d import Axes3D
from sklearn import manifold



warnings.filterwarnings('ignore')

In [44]:
xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

xtrain.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,118345.392781,3104.010883,82378.542430,1086.678601,11395.883550,10.455601,109886.476608,1.054926e+06,101877.946528,2.550290,...,10.769524,10.245068,1021.907543,1031.645762,107440.039766,107083.296966,5559.203921,10.861464,107548.413675,2.299202
1,NaN,5889.055991,97932.067482,1116.564363,12172.212426,10.502997,100372.504488,1.011783e+06,108852.296962,2.579040,...,10.859822,10.088023,988.135968,999.044639,105238.779040,106365.468525,10334.091649,NaN,107841.227180,2.472763
2,95373.022035,6293.442455,93371.369288,1034.722073,11618.142107,10.947611,109337.757279,1.006491e+06,104540.883376,2.573247,...,10.356015,10.514234,995.480813,898.329758,107044.005199,102704.104686,7697.875887,10.277760,103414.594212,NaN
3,105066.272301,3279.162164,119554.079514,1114.014483,12127.235755,10.540204,106215.391332,1.022381e+06,109597.474506,2.693256,...,10.609811,10.991861,1117.179063,867.267810,100617.082858,108893.304959,6549.131099,9.337688,105722.857008,2.484935
4,98957.033333,4177.482190,97239.708338,1049.430921,10417.250336,10.618957,102736.470171,1.086871e+06,103340.057448,2.540610,...,10.397777,NaN,863.943966,1041.278922,105352.672454,101705.692839,5388.312773,12.196382,NaN,1.930136


In [45]:
# Treat missing values as column medians. Important (apparently) to use the medians from the training set in the test set

xtrain = xtrain.fillna(xtrain.median())
xtest = xtest.fillna(xtrain.median())

In [46]:
#reshuffle data 

xtrain = xtrain.sample(frac=1)
xtrain.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
226,82460.328787,4741.750443,111723.486005,1012.756760,10752.861742,10.687922,103775.740202,1.074283e+06,106350.241524,2.441967,...,10.796367,10.337088,915.133140,1027.732504,107387.600648,109788.839355,5739.344254,8.899849,102054.008814,2.180407
902,111511.044191,3668.027201,113368.354058,999.260914,9615.481502,10.674596,103053.672255,1.049304e+06,106108.816396,2.510628,...,10.606812,10.815623,851.699490,973.962309,105011.047274,104017.336143,5293.301027,10.928300,106075.158782,2.086920
1094,108573.930975,3473.952973,104267.518211,999.260914,10057.298525,10.328210,107032.120612,1.088016e+06,107915.593155,2.390152,...,10.895187,10.540194,1059.954416,1079.015511,101696.442454,109230.629277,8316.689407,9.135381,109907.113429,2.226447
484,94688.541971,4802.652371,105730.982822,1117.741875,10018.714253,10.177280,107497.530666,1.032781e+06,101102.037961,2.361084,...,10.339632,10.538567,1030.570501,770.848212,100894.897139,108192.956873,5556.710750,11.269456,103903.219714,2.011302
183,94557.166771,3646.820066,102673.739589,934.258782,8176.660307,10.858726,105867.565756,1.080882e+06,105692.993988,2.499077,...,10.691278,10.375133,1097.991955,1081.138168,103972.024560,108961.923942,7536.352608,10.539626,104474.347498,2.301115


In [47]:
# Scale the training and test data

scaler = preprocessing.StandardScaler()

xtrain_scaled = scaler.fit_transform(xtrain)
xtrain = pd.DataFrame(xtrain_scaled, columns = xtrain.columns)

xtest_scaled = scaler.transform(xtest)
xtest = pd.DataFrame(xtest_scaled, columns = xtest.columns)

In [48]:
#id column from ytrain 
ytrain1 = ytrain.loc[:, "y"]
ytrain2 = pd.DataFrame(data = ytrain1.values, columns= ['y'])
#ytrain2.head()
ytrain = ytrain2
ytrain.head()

,y
0,75.0
1,53.0
2,78.0
3,65.0
4,86.0


In [51]:
#Lasso

x = xtrain.values
y = ytrain.values

kf = KFold(n_splits=10, shuffle = True)

alphas = np.array([0.1, 0.2, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.35, 0.4, 1, 3, 10])
meanscores = []
estimators = 10
for train_index, test_index in kf.split(x):
    X_train = x[train_index]
    X_test = x[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    sel = SelectFromModel(Lasso(alpha=0.3))
    sel.fit(X_train, y_train)
    X_train_sel = sel.transform(X_train)
    X_test_sel = sel.transform(X_test)

    print("Selected training {} &&& Selected Test {}".format(X_train_sel.shape, X_test_sel.shape))

    clf = RandomForestRegressor(n_estimators=50)
    clf.fit(X_train_sel, y_train)

    y_pred = clf.predict(X_test_sel)
    # print(y_pred,y_test)
    print(r2_score(y_test, y_pred))



Selected training (1090, 174) &&& Selected Test (122, 174)
-0.0263760226238825
Selected training (1090, 181) &&& Selected Test (122, 181)
-0.14777680648387337
Selected training (1091, 169) &&& Selected Test (121, 169)


KeyboardInterrupt: 